In [ ]:
from graph_var.utils import *
from graph_var.graph import PangenomeGraph
from graph_var.evaluating_functions import *
import pandas as pd

In [ ]:
graph_obj_dir = './Graph_objs'
raw_vcf_dir = './VCFs_chr'
sub_vcf_dir = './VCFs_subset'
graph_vcf_dir = './VCFs_update'
stats_dir = './Stats_update'
ac_dir = './Bubble_result_update'
tree_dir = './Tree'
region_dir = './Region_files'
bubble_summary_dir = './Bubble_summary'
bubble_result_dir = './Bubble_result_update'

In [ ]:
def summarise_walk(G, walk):
    edge_dict = {}
    for edge in zip(walk[:-1], walk[1:]):
        if G.edges[edge]['is_in_tree']:
            edge_dict['in_tree'] = edge_dict.get('in_tree', 0) + 1
        else:
            if not G.edges[edge]['is_representative']:
                edge = edge_complement(edge)
            var_type = G.identify_variant_type(edge)
            edge_dict[var_type] = edge_dict.get(var_type, 0) + 1
    return {k: edge_dict[k] for k in sorted(edge_dict.keys())}

Check if inversion or repeat exist near CYP2D6 gene

In [ ]:
graph_path = f"{graph_obj_dir}/chr22.pkl"
G, walks, sample_names = load_graph_from_pkl(graph_path)
sample_walks_dict = group_walks_by_name(walks, sample_names)
HG01071_genotypes = [G.genotype(walk) for walk in sample_walks_dict['HG01071_2']]

In [ ]:
len(HG01071_genotypes), len(sample_walks_dict['HG01071_2'])

In [ ]:
tree_ref_edge = G.reference_tree_edge(('44987811_-', '44987812_+'))
tree_ref_edge

In [ ]:
[HG01071_genotypes[i][0].get(tree_ref_edge, 0) for i in range(len(HG01071_genotypes))]

In [ ]:
[(idx+1, summarise_walk(G, walk)) for idx, walk in enumerate(sample_walks_dict['HG01071_2'])]